# Horse Racing Prediction by Deep Learning

This AI Horse racing prediction is using a 3-layer Neural Network to predict the finishing position of each horse in a race. The data used to train this model is obtained from the Hong Kong Jockey Club (HKJC). The model is built and trained by Tensorflow. Supervised learning is used in this project to classify the expected finishing position of the horses.

<H2> Part 1: Data Input and Preprocessing </H2>

In this part of the program, we will import the data obtained from the HKJC. First of all, the following features were selected based on my past horse picking experience, namely:

- position: The starting position of the horse. If the position is "1", it indicates the closest position to the hurdle and should be benficial in non-straight race courses.

- load: This is the loading of the horse in pounds. Maximum is 133.

- ON odds: This is the overnight odds of the horse provided by the HKJC.

- odds: This is the odds of the horse 15 min before the race.

- class: This is the class of the case. It is common to all horses in a race except special races.

- num horses: This is the number of horses participated in the race.


In [1]:
#Loading the data and preprocessing it.

import pandas as pd
import tensorflow as tf
#tf.__version__

PATH_TRAINING_DATA = 'training_data/horse_data_train_test.csv'

dataset = pd.read_csv(PATH_TRAINING_DATA)
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle=False)


In [2]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train) #use "fit_transform" for training data
X_test = sc.transform(X_test)       #use "transform" for testing data

y_train = tf.keras.utils.to_categorical((y_train-1), 14)
y_test = tf.keras.utils.to_categorical((y_test-1), 14)

print(y_train)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


<H2> Part 2: Building the Model </H2>

We will construct a basic 3-layer model: one input layer, one output layer and one hidden layer.
I have tried relu and sigmoid as the activation, and found that sigmoid seems to be a better choice for this project.


In [3]:
#create a new model
def build_model():

    # Initializing the ANN
    model = tf.keras.models.Sequential()

    # Adding the input layer and the first layer
    # We have six features now, and one bias term, so the input layer has a size of 6+1
    #model.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=7, activation='sigmoid'))
    #model.add(tf.keras.layers.Dense(units=7, activation='relu'))

    # Adding the input layer and the hidden layer
    # We set the size of the hidden layer to be the mean of input and output later, i.e. 10
    #model.add(tf.keras.layers.Dense(units=5, activation='sigmoid')) 
    #model.add(tf.keras.layers.Dense(units=6, activation='sigmoid')) 
    #model.add(tf.keras.layers.Dense(units=10, activation='sigmoid')) 
    #model.add(tf.keras.layers.Dense(units=10, activation='relu')) 
    model.add(tf.keras.layers.Dense(units=6, activation='sigmoid')) 

    # Adding a drop out layer
    #model.add(tf.keras.layers.Dropout(0.2))
    #model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dropout(0.35277))

    #model.add(tf.keras.layers.Dense(units=6, activation='sigmoid')) 
    #model.add(tf.keras.layers.Dropout(0.2))

    # Adding the output layer
    # We have 14 outputs, so the output later has a size of 14
    model.add(tf.keras.layers.Dense(units=14, activation='softmax'))

    # Compiling the model
    #opt = tf.keras.optimizers.Adam(learning_rate=0.03) 
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
    return model

model = build_model() 

<H2> Part 3a: [Optional] Training and saving a new Model </H2>

In this section, we would either traing the model from scratch or just load a pre-trained model that is shipped with this Jupyter notebook.

- If the training data has been changed or the model has been renewed, go to train a new model in code cell 3a)
- If you want to save some time, just skip cell 3a) and load a pre-trained model in code cell 3b)


In [5]:
#This is code cell 3a) that trains a model from scratch and then saves it. Please note that it takes hours to train!

import tensorflow as tf
import datetime, os

%load_ext tensorboard
#%reload_ext tensorboard

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

print("rows in X_train = " + str(X_train.shape[0]) )
print("rows in y_train = " + str(y_train.shape[0]) )

print("Training model...")
model.fit(x=X_train, 
          y=y_train, 
          batch_size = 14, 
          epochs = 5000, #10, #10000, #30000 #50000 #20000
          validation_data=(X_test, y_test), 
          callbacks=[tensorboard_callback])

# save the model for later use
print("Saving model...")
model.save('saved_model/my_model')

print("Model saved!")

rows in X_train = 491
rows in y_train = 491
Training model...
Epoch 1/5000
36/36 [==============================] - 2s 24ms/step - loss: 2.6962 - accuracy: 0.0754 - val_loss: 2.6211 - val_accuracy: 0.0976
Epoch 2/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.6560 - accuracy: 0.0407 - val_loss: 2.6185 - val_accuracy: 0.0976
Epoch 3/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.6248 - accuracy: 0.0896 - val_loss: 2.6014 - val_accuracy: 0.0976
Epoch 4/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.6148 - accuracy: 0.0916 - val_loss: 2.5809 - val_accuracy: 0.1138
Epoch 5/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.5858 - accuracy: 0.1161 - val_loss: 2.5402 - val_accuracy: 0.1301
Epoch 6/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.5499 - accuracy: 0.1324 - val_loss: 2.5034 - val_accuracy: 0.1301
Epoch 7/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.5066 - ac

Epoch 58/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.3253 - accuracy: 0.1609 - val_loss: 2.2963 - val_accuracy: 0.1707
Epoch 59/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.2874 - accuracy: 0.1711 - val_loss: 2.2967 - val_accuracy: 0.1545
Epoch 60/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.2941 - accuracy: 0.1548 - val_loss: 2.3105 - val_accuracy: 0.1789
Epoch 61/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.3307 - accuracy: 0.1731 - val_loss: 2.2974 - val_accuracy: 0.1463
Epoch 62/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.3334 - accuracy: 0.1507 - val_loss: 2.3029 - val_accuracy: 0.1789
Epoch 63/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.3299 - accuracy: 0.1711 - val_loss: 2.3077 - val_accuracy: 0.1870
Epoch 64/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.3120 - accuracy: 0.1853 - val_loss: 2.3080 - val_accuracy: 0.1870

Epoch 172/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.2551 - accuracy: 0.1935 - val_loss: 2.2916 - val_accuracy: 0.1789
Epoch 173/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.2521 - accuracy: 0.2016 - val_loss: 2.2880 - val_accuracy: 0.1707
Epoch 174/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.2592 - accuracy: 0.1670 - val_loss: 2.2946 - val_accuracy: 0.1951
Epoch 175/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.2559 - accuracy: 0.1752 - val_loss: 2.2902 - val_accuracy: 0.1951
Epoch 176/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.2788 - accuracy: 0.1690 - val_loss: 2.2900 - val_accuracy: 0.1951
Epoch 177/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.2257 - accuracy: 0.1935 - val_loss: 2.3030 - val_accuracy: 0.2114
Epoch 178/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.2497 - accuracy: 0.1874 - val_loss: 2.2852 - val_accuracy:

Epoch 229/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.2465 - accuracy: 0.1996 - val_loss: 2.2732 - val_accuracy: 0.1951
Epoch 230/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.2075 - accuracy: 0.1813 - val_loss: 2.2786 - val_accuracy: 0.1951
Epoch 231/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.2398 - accuracy: 0.1914 - val_loss: 2.2797 - val_accuracy: 0.1951
Epoch 232/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.2420 - accuracy: 0.1894 - val_loss: 2.2661 - val_accuracy: 0.2033
Epoch 233/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.2188 - accuracy: 0.2098 - val_loss: 2.2690 - val_accuracy: 0.2114
Epoch 234/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.2378 - accuracy: 0.1976 - val_loss: 2.2784 - val_accuracy: 0.1870
Epoch 235/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.2275 - accuracy: 0.1996 - val_loss: 2.2828 - val_accuracy:

Epoch 342/5000
36/36 [==============================] - 0s 6ms/step - loss: 2.1649 - accuracy: 0.2179 - val_loss: 2.2793 - val_accuracy: 0.1951
Epoch 343/5000
36/36 [==============================] - 0s 5ms/step - loss: 2.1850 - accuracy: 0.2301 - val_loss: 2.2866 - val_accuracy: 0.1870
Epoch 344/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.1839 - accuracy: 0.1935 - val_loss: 2.2844 - val_accuracy: 0.2114
Epoch 345/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.1830 - accuracy: 0.2301 - val_loss: 2.2891 - val_accuracy: 0.1951
Epoch 346/5000
36/36 [==============================] - 0s 5ms/step - loss: 2.2167 - accuracy: 0.2016 - val_loss: 2.2864 - val_accuracy: 0.1707
Epoch 347/5000
36/36 [==============================] - 0s 5ms/step - loss: 2.1742 - accuracy: 0.2179 - val_loss: 2.2894 - val_accuracy: 0.1951
Epoch 348/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.1845 - accuracy: 0.2118 - val_loss: 2.2878 - val_accuracy:

...

36/36 [==============================] - 0s 3ms/step - loss: 2.0761 - accuracy: 0.2607 - val_loss: 2.7839 - val_accuracy: 0.1870
Epoch 4498/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.0489 - accuracy: 0.2627 - val_loss: 2.8769 - val_accuracy: 0.1626
Epoch 4499/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0631 - accuracy: 0.2648 - val_loss: 2.8441 - val_accuracy: 0.1707
Epoch 4500/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0341 - accuracy: 0.2403 - val_loss: 2.8629 - val_accuracy: 0.1626
Epoch 4501/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.0257 - accuracy: 0.2811 - val_loss: 2.9002 - val_accuracy: 0.1545
Epoch 4502/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.0527 - accuracy: 0.2464 - val_loss: 2.9439 - val_accuracy: 0.1789
Epoch 4503/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0170 - accuracy: 0.2953 - val_loss: 2.8997 - val_accuracy: 0.1870
E

36/36 [==============================] - 0s 4ms/step - loss: 2.0728 - accuracy: 0.2525 - val_loss: 2.9065 - val_accuracy: 0.1545
Epoch 4610/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0427 - accuracy: 0.2607 - val_loss: 2.9569 - val_accuracy: 0.1463
Epoch 4611/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0004 - accuracy: 0.2933 - val_loss: 2.9970 - val_accuracy: 0.1626
Epoch 4612/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0752 - accuracy: 0.2464 - val_loss: 2.9719 - val_accuracy: 0.1626
Epoch 4613/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0442 - accuracy: 0.2363 - val_loss: 2.9222 - val_accuracy: 0.1463
Epoch 4614/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0747 - accuracy: 0.2525 - val_loss: 2.8790 - val_accuracy: 0.1545
Epoch 4615/5000
36/36 [==============================] - 0s 4ms/step - loss: 2.0736 - accuracy: 0.2444 - val_loss: 2.8621 - val_accuracy: 0.1545
E

36/36 [==============================] - 0s 3ms/step - loss: 2.0345 - accuracy: 0.2485 - val_loss: 2.9278 - val_accuracy: 0.1463
Epoch 4722/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0816 - accuracy: 0.2464 - val_loss: 2.9510 - val_accuracy: 0.1545
Epoch 4723/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0286 - accuracy: 0.2648 - val_loss: 2.8468 - val_accuracy: 0.1463
Epoch 4724/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0714 - accuracy: 0.2322 - val_loss: 2.9754 - val_accuracy: 0.1382
Epoch 4725/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0683 - accuracy: 0.2546 - val_loss: 2.9500 - val_accuracy: 0.1626
Epoch 4726/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0505 - accuracy: 0.2546 - val_loss: 2.9395 - val_accuracy: 0.1463
Epoch 4727/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0502 - accuracy: 0.2566 - val_loss: 2.9742 - val_accuracy: 0.1707
E

36/36 [==============================] - 0s 3ms/step - loss: 2.0336 - accuracy: 0.2729 - val_loss: 2.9120 - val_accuracy: 0.1463
Epoch 4834/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0246 - accuracy: 0.2587 - val_loss: 2.9931 - val_accuracy: 0.1545
Epoch 4835/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0393 - accuracy: 0.2607 - val_loss: 2.9744 - val_accuracy: 0.1382
Epoch 4836/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0637 - accuracy: 0.2648 - val_loss: 2.9412 - val_accuracy: 0.1382
Epoch 4837/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0293 - accuracy: 0.2688 - val_loss: 2.9515 - val_accuracy: 0.1382
Epoch 4838/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0641 - accuracy: 0.2505 - val_loss: 2.8892 - val_accuracy: 0.1382
Epoch 4839/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0624 - accuracy: 0.2729 - val_loss: 2.9666 - val_accuracy: 0.1545
E

36/36 [==============================] - 0s 3ms/step - loss: 2.0744 - accuracy: 0.2444 - val_loss: 2.9750 - val_accuracy: 0.1463
Epoch 4946/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0274 - accuracy: 0.2607 - val_loss: 2.9999 - val_accuracy: 0.1545
Epoch 4947/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0199 - accuracy: 0.2587 - val_loss: 3.0496 - val_accuracy: 0.1626
Epoch 4948/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0134 - accuracy: 0.2790 - val_loss: 3.0186 - val_accuracy: 0.1789
Epoch 4949/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0343 - accuracy: 0.2587 - val_loss: 3.0303 - val_accuracy: 0.1463
Epoch 4950/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0417 - accuracy: 0.2546 - val_loss: 2.9374 - val_accuracy: 0.1545
Epoch 4951/5000
36/36 [==============================] - 0s 3ms/step - loss: 2.0539 - accuracy: 0.2627 - val_loss: 3.0207 - val_accuracy: 0.1707
E

<H2> Part 3b: Loading a pre-trained Model </H2>

If you have ever run cell 3a), you will load the model that you have trained. Otherwise, you will load a pre-trained model that is shipped with this Jupyter notebook.

In [4]:
#This is code cell 3b) that loads a pre-trained model. 

model = tf.keras.models.load_model('saved_model/my_model')

print("model is loaded")

model is loaded


<H2>Part 3c: Model Summary</H2>
    
Let's take a look on the model:

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 7)                 49        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 48        
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                98        
Total params: 195
Trainable params: 195
Non-trainable params: 0
_________________________________________________________________


<H2> Part 4: Evaluation </H2>

This part calculates the confusion matrix and accuracy_score of the model.

In [6]:
# Restore the weights
import numpy as np
import pandas as pd

dataset = pd.read_csv('training_data/horse_data_train_test.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


import tensorflow as tf
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Predicting the Test set results
y_pred = model.predict( X_test )  #y_pred = new_model.predict( sc.fit_transform(X_test) ) 
y_pred = np.argmax(y_pred, axis = 1)

In [7]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, (y_pred+1))
print("cm=", cm)
print("accuracy_score=", accuracy_score(y_test, (y_pred+1)))

cm= [[0 9 2 0 0 0 1 0 0 0 0 0 0 0]
 [1 5 1 0 1 0 0 1 0 0 0 0 0 0]
 [0 5 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 2 0 1 0 3 0 0 0 0 0 0 0]
 [0 1 0 0 5 0 1 2 0 0 0 0 0 1]
 [1 3 0 0 1 0 2 0 0 0 0 0 0 0]
 [0 5 0 0 2 0 3 2 0 0 0 0 0 0]
 [0 4 0 0 2 0 0 1 1 0 0 1 0 0]
 [2 1 0 0 2 0 1 1 1 0 0 0 0 1]
 [0 0 1 0 4 0 0 3 0 1 0 1 0 0]
 [0 0 1 0 2 0 0 1 0 0 1 2 2 0]
 [0 0 0 0 2 0 2 1 0 1 1 1 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 2 4 1]
 [0 0 0 0 0 0 1 0 0 0 0 0 1 7]]
accuracy_score= 0.234375


<H2> Part 5: [Optional] Visualization of the model by Tensorboard </H2>

***Note:*** Run this section only if you have run Part3a.

By making use of the powerful visual tools given by Tensorboard, we can tune the hyper parameters of the model and visualize the results easily. If the epoch_accuracy is increasing steadily with epoch for the validation set, we could be confident that the model is on track. 

If we are not satisfy with the performaces shown in the Tensorboard, we can go back to refine the data processing, model building and hyper parameters and check on Tensorboard again iteratively.

In [10]:
print("Calling tensorboard...")

#import datetime, os
#%load_ext tensorboard
#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

%tensorboard --logdir logs --port=6008 #use a different port if tensorboard fails to load!


Calling tensorboard...


Reusing TensorBoard on port 6008 (pid 8956), started 5 days, 5:23:31 ago. (Use '!kill 8956' to kill it.)

<H2> Part 6: Prediction Checking </H2>

In this section, we will try to predict the results of a race on 2021/09/22, and see how accurate the model is by comparing the rediction witht the real results.
 

In [8]:
#This is the csv containing the data of a new race:

#CURRENT_RACE_DATA = 'new_data/horse_data_20210926_race8.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20210926_race3.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20210926_race10.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20211001_race2.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20211001_race3.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20211001_race7.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20211001_race9.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20211006_race2.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20211010_race2.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20211017_race6.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20211017_race8.csv'
#CURRENT_RACE_DATA = 'new_data/horse_data_20211020_race3.csv'
CURRENT_RACE_DATA = 'new_data/horse_data_20211020_race5.csv'

#CURRENT_RACE_DATA = 'new_data/horse_data_20210922_race3.csv'
REAL_FINISHING_POSITIONS = '[ 1  6  4  2 12 10  8  3  7  5 11  9 13 14]'

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

PATH_TRAINING_DATA = 'training_data/horse_data_train_test.csv'

#training data
dataset = pd.read_csv(PATH_TRAINING_DATA)
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0, shuffle=False)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)

#live data
dataset = pd.read_csv(CURRENT_RACE_DATA)
X_live = dataset.iloc[:, :-1].values

#use "transform" for live data
X_live = sc.transform(X_live)

#loading pretrained model
model = tf.keras.models.load_model('saved_model/my_model')

# Predicting the Test set results
y_pred = model.predict(X_live)  

# Predicting the finishing position
y_pred_finishing = np.argmax(y_pred, axis = 1) 
print("Expected finishing positions=", y_pred_finishing+1)

print("The real finishing positions= " + REAL_FINISHING_POSITIONS)


Expected finishing positions= [5 2 5 8 8 2 2 1 5 2 5 5]
The real finishing positions= [ 1  6  4  2 12 10  8  3  7  5 11  9 13 14]


For prediction on your own, you need to create a new csv file that contains the data of a new race in the folder new_data, then run the preditction.